# Dependencies

In [2]:
!pip install tensorflow tensorflow-gpu opencv-python mediapipe sklearn matplotlib

In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Hand, Face and Pose Keypoints

In [4]:
mp_holistic = mp.solutions.holistic # holistic models
mp_drawing = mp.solutions.drawing_utils #drawing utilities

In [5]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # BGR 2 RGB
    image.flags.writeable = False # Not writable
    results = model.process(image) # make prediction
    image.flags.writeable = True # make writable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [6]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [7]:
def draw_styled_landmarks(image,results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110,10),thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121),thickness=1, circle_radius=1)
                             )
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10),thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121),thickness=2, circle_radius=2)
                             )
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76),thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250),thickness=2, circle_radius=2)
                             )
    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66),thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245,66,230),thickness=2, circle_radius=2)
                             )

In [8]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        ret,frame = cap.read()
        
        image, results = mediapipe_detection(frame,holistic)
        print(results)
        
        draw_styled_landmarks(image,results)
        
        cv2.imshow('OpenCV Feed',image)
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [9]:
draw_landmarks(frame,results)

# Extracting Keypoints

In [11]:
pose = []
for res in results.pose_landmarks.landmark:
    test=np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [12]:
def extract_keypoints(results):
    pose = np.array([[res.x,res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(32*4)
    face = np.array([[res.x,res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x,res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x,res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])

In [15]:
result_test = extract_keypoints(results)
np.save('0', result_test)
np.load('0.npy')

array([ 0.81692266,  0.74183089, -1.76392865, ...,  0.        ,
        0.        ,  0.        ])

# Data Folders

In [16]:
DATA_PATH = os.path.join('MP_DATA')

#actions to detect
actions = np.array(['hello','thanks','iloveyou'])
# 30 videos of data
no_sequence = 30
# 30 frames
sequence_length = 30

In [ ]:
# hello
##0
##1
##2
##29
#thanks
#iloveyou

In [17]:
for action in actions:
    for sequence in range(no_sequence):
        try:
            # hello - frameno
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Collect Data

In [ ]:
# cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    
    # loop through the actions
    for action in actions:
        #loop through sequence of videos
        for sequence in range(no_sequence):
            #loop  through each frame
            for frame_num in range(sequence_length):
                ret,frame = cap.read()

                image, results = mediapipe_detection(frame,holistic)
                print(results)

                draw_styled_landmarks(image,results)
                
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION',(120,200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} video number {}'.format(action,sequence),(15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed',image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {} video number {}'.format(action,sequence),(15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed',image)
                    
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                
                if cv2.waitKey(10) & 0xFF==ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

In [19]:
cap.release()
cv2.destroyAllWindows()

# Preprocess Data and Create Labels and Features

In [21]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [22]:
label_map = {label:num for num, label in enumerate(actions)}

In [23]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [25]:
sequences,labels = [],[]

for action in actions:
    for sequence in range(no_sequence):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
    

In [26]:
np.array(sequences).shape

(90, 30, 1662)

In [27]:
np.array(labels).shape

(90,)

In [28]:
X = np.array(sequences)

In [29]:
y = to_categorical(labels).astype(int)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.05)

#  MODEL

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [44]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir = log_dir)

In [58]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation = 'relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation = 'relu'))
model.add(LSTM(64, return_sequences=False, activation = 'relu'))
model.add(Dense(64,activation = 'relu'))
model.add(Dense(32,activation = 'relu'))
model.add(Dense(actions.shape[0], activation = 'softmax'))

In [59]:
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics=["categorical_accuracy"])

In [47]:
model.fit(X_train,y_train, epochs = 2000, callbacks = [tb_callback])

Epoch 1/2000
3/3 [==============================] - 11s 190ms/step - loss: 1.8197 - categorical_accuracy: 0.3882
Epoch 2/2000
3/3 [==============================] - 1s 186ms/step - loss: 7.9970 - categorical_accuracy: 0.3647
Epoch 3/2000
3/3 [==============================] - 1s 196ms/step - loss: 2.5489 - categorical_accuracy: 0.2706
Epoch 4/2000
3/3 [==============================] - 1s 177ms/step - loss: 2.0650 - categorical_accuracy: 0.4000
Epoch 5/2000
3/3 [==============================] - 0s 108ms/step - loss: 1.2540 - categorical_accuracy: 0.4824
Epoch 6/2000
3/3 [==============================] - 0s 116ms/step - loss: 1.0126 - categorical_accuracy: 0.5765
Epoch 7/2000
3/3 [==============================] - 0s 111ms/step - loss: 0.7417 - categorical_accuracy: 0.6000
Epoch 8/2000
3/3 [==============================] - 0s 123ms/step - loss: 0.6951 - categorical_accuracy: 0.6235
Epoch 9/2000
3/3 [==============================] - 0s 118ms/step - loss: 0.6115 - categorical_accuracy

3/3 [==============================] - 0s 118ms/step - loss: 0.4126 - categorical_accuracy: 0.7765
Epoch 74/2000
3/3 [==============================] - 0s 134ms/step - loss: 0.3887 - categorical_accuracy: 0.7647
Epoch 75/2000
3/3 [==============================] - 0s 140ms/step - loss: 0.3644 - categorical_accuracy: 0.8471
Epoch 76/2000
3/3 [==============================] - 0s 118ms/step - loss: 0.3501 - categorical_accuracy: 0.8118
Epoch 77/2000
3/3 [==============================] - 0s 135ms/step - loss: 0.3511 - categorical_accuracy: 0.8118
Epoch 78/2000
3/3 [==============================] - 1s 149ms/step - loss: 0.3388 - categorical_accuracy: 0.8235
Epoch 79/2000
3/3 [==============================] - 0s 138ms/step - loss: 0.2812 - categorical_accuracy: 0.8824
Epoch 80/2000
3/3 [==============================] - 0s 140ms/step - loss: 0.2793 - categorical_accuracy: 0.8706
Epoch 81/2000
3/3 [==============================] - 0s 143ms/step - loss: 0.2765 - categorical_accuracy: 0.89

3/3 [==============================] - 0s 148ms/step - loss: 0.1616 - categorical_accuracy: 0.9765
Epoch 146/2000
3/3 [==============================] - 0s 111ms/step - loss: 0.1944 - categorical_accuracy: 0.9176
Epoch 147/2000
3/3 [==============================] - 0s 121ms/step - loss: 0.1551 - categorical_accuracy: 0.9294
Epoch 148/2000
3/3 [==============================] - 0s 117ms/step - loss: 0.1281 - categorical_accuracy: 0.9765
Epoch 149/2000
3/3 [==============================] - 0s 118ms/step - loss: 0.1392 - categorical_accuracy: 0.9529
Epoch 150/2000
3/3 [==============================] - 0s 110ms/step - loss: 0.0919 - categorical_accuracy: 0.9765
Epoch 151/2000
3/3 [==============================] - 0s 110ms/step - loss: 0.0682 - categorical_accuracy: 0.9882
Epoch 152/2000
3/3 [==============================] - 0s 145ms/step - loss: 0.0664 - categorical_accuracy: 0.9765
Epoch 153/2000
3/3 [==============================] - 0s 133ms/step - loss: 0.0708 - categorical_accura

3/3 [==============================] - 0s 144ms/step - loss: 1.9310e-04 - categorical_accuracy: 1.0000
Epoch 217/2000
3/3 [==============================] - 0s 117ms/step - loss: 1.8797e-04 - categorical_accuracy: 1.0000
Epoch 218/2000
3/3 [==============================] - 0s 142ms/step - loss: 1.8378e-04 - categorical_accuracy: 1.0000
Epoch 219/2000
3/3 [==============================] - 0s 124ms/step - loss: 1.7862e-04 - categorical_accuracy: 1.0000
Epoch 220/2000
3/3 [==============================] - 0s 126ms/step - loss: 1.7442e-04 - categorical_accuracy: 1.0000
Epoch 221/2000
3/3 [==============================] - 0s 126ms/step - loss: 1.7133e-04 - categorical_accuracy: 1.0000
Epoch 222/2000
3/3 [==============================] - 0s 99ms/step - loss: 1.6675e-04 - categorical_accuracy: 1.0000
Epoch 223/2000
3/3 [==============================] - 0s 104ms/step - loss: 1.6250e-04 - categorical_accuracy: 1.0000
Epoch 224/2000
3/3 [==============================] - 0s 125ms/step - lo

3/3 [==============================] - 0s 156ms/step - loss: 6.3425e-05 - categorical_accuracy: 1.0000
Epoch 286/2000
3/3 [==============================] - 0s 131ms/step - loss: 6.2654e-05 - categorical_accuracy: 1.0000
Epoch 287/2000
3/3 [==============================] - 0s 140ms/step - loss: 6.1897e-05 - categorical_accuracy: 1.0000
Epoch 288/2000
3/3 [==============================] - 0s 141ms/step - loss: 6.1119e-05 - categorical_accuracy: 1.0000
Epoch 289/2000
3/3 [==============================] - 1s 152ms/step - loss: 6.0555e-05 - categorical_accuracy: 1.0000
Epoch 290/2000
3/3 [==============================] - 0s 130ms/step - loss: 6.0014e-05 - categorical_accuracy: 1.0000
Epoch 291/2000
3/3 [==============================] - 0s 130ms/step - loss: 5.9378e-05 - categorical_accuracy: 1.0000
Epoch 292/2000
3/3 [==============================] - 0s 133ms/step - loss: 5.8775e-05 - categorical_accuracy: 1.0000
Epoch 293/2000
3/3 [==============================] - 0s 135ms/step - l

...

3/3 [==============================] - 0s 110ms/step - loss: 2.0336e-06 - categorical_accuracy: 1.0000
Epoch 1045/2000
3/3 [==============================] - 0s 108ms/step - loss: 2.0293e-06 - categorical_accuracy: 1.0000
Epoch 1046/2000
3/3 [==============================] - 0s 111ms/step - loss: 2.0223e-06 - categorical_accuracy: 1.0000
Epoch 1047/2000
3/3 [==============================] - 0s 119ms/step - loss: 2.0195e-06 - categorical_accuracy: 1.0000
Epoch 1048/2000
3/3 [==============================] - 0s 109ms/step - loss: 2.0153e-06 - categorical_accuracy: 1.0000
Epoch 1049/2000
3/3 [==============================] - 0s 113ms/step - loss: 2.0055e-06 - categorical_accuracy: 1.0000
Epoch 1050/2000
3/3 [==============================] - 0s 164ms/step - loss: 2.0013e-06 - categorical_accuracy: 1.0000
Epoch 1051/2000
3/3 [==============================] - 0s 152ms/step - loss: 1.9887e-06 - categorical_accuracy: 1.0000
Epoch 1052/2000
3/3 [==============================] - 0s 142ms/

3/3 [==============================] - 0s 136ms/step - loss: 1.7026e-06 - categorical_accuracy: 1.0000
Epoch 1114/2000
3/3 [==============================] - 0s 135ms/step - loss: 1.7012e-06 - categorical_accuracy: 1.0000
Epoch 1115/2000
3/3 [==============================] - 0s 136ms/step - loss: 1.6970e-06 - categorical_accuracy: 1.0000
Epoch 1116/2000
3/3 [==============================] - 0s 142ms/step - loss: 1.6942e-06 - categorical_accuracy: 1.0000
Epoch 1117/2000
3/3 [==============================] - 0s 144ms/step - loss: 1.6886e-06 - categorical_accuracy: 1.0000
Epoch 1118/2000
3/3 [==============================] - 1s 219ms/step - loss: 1.6829e-06 - categorical_accuracy: 1.0000
Epoch 1119/2000
3/3 [==============================] - 0s 130ms/step - loss: 1.6787e-06 - categorical_accuracy: 1.0000
Epoch 1120/2000
3/3 [==============================] - 0s 158ms/step - loss: 1.6773e-06 - categorical_accuracy: 1.0000
Epoch 1121/2000
3/3 [==============================] - 0s 140ms/

Epoch 1182/2000
3/3 [==============================] - 0s 131ms/step - loss: 1.4571e-06 - categorical_accuracy: 1.0000
Epoch 1183/2000
3/3 [==============================] - 0s 141ms/step - loss: 1.4515e-06 - categorical_accuracy: 1.0000
Epoch 1184/2000
3/3 [==============================] - 0s 128ms/step - loss: 1.4501e-06 - categorical_accuracy: 1.0000
Epoch 1185/2000
3/3 [==============================] - 0s 128ms/step - loss: 1.4445e-06 - categorical_accuracy: 1.0000
Epoch 1186/2000
3/3 [==============================] - 0s 127ms/step - loss: 1.4389e-06 - categorical_accuracy: 1.0000
Epoch 1187/2000
3/3 [==============================] - 0s 119ms/step - loss: 1.4347e-06 - categorical_accuracy: 1.0000
Epoch 1188/2000
3/3 [==============================] - 0s 118ms/step - loss: 1.4333e-06 - categorical_accuracy: 1.0000
Epoch 1189/2000
3/3 [==============================] - 0s 146ms/step - loss: 1.4277e-06 - categorical_accuracy: 1.0000
Epoch 1190/2000
3/3 [===========================

3/3 [==============================] - 0s 142ms/step - loss: 1.2496e-06 - categorical_accuracy: 1.0000
Epoch 1251/2000
3/3 [==============================] - 0s 130ms/step - loss: 1.2454e-06 - categorical_accuracy: 1.0000
Epoch 1252/2000
1/3 [=========>....................] - ETA: 0s - loss: 1.8403e-06 - categorical_accuracy: 1.0000

KeyboardInterrupt: 

# Predict

In [48]:
res = model.predict(X_test)

1/1 [==============================] - 2s 2s/step


In [49]:
actions[np.argmax(res[3])]

'thanks'

In [55]:
actions[np.argmax(y_test[3])]

'thanks'

# Save weights

In [56]:
model.save('action.h5')

In [57]:
del model

In [61]:
model.load_weights('action.h5')

# Evalution

In [62]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [68]:
yhat = model.predict(X_train)

3/3 [==============================] - 0s 53ms/step


In [69]:
y_true = np.argmax(y_train, axis = 1).tolist()
yhat = np.argmax(yhat, axis = 1).tolist()

In [70]:
multilabel_confusion_matrix(y_true,yhat)

array([[[56,  0],
        [ 0, 29]],

       [[59,  0],
        [ 0, 26]],

       [[55,  0],
        [ 0, 30]]], dtype=int64)

In [71]:
accuracy_score(y_true,yhat)

1.0

# test in real time

In [76]:
sequence = []
sentence = []
threshold = 0.4

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        ret,frame = cap.read()
        
        image, results = mediapipe_detection(frame,holistic)
        print(results)
        
        draw_styled_landmarks(image,results)
        
        keypoints = extract_keypoints(results)
        sequence.insert(0,keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
        
        if res[np.argmax(res)] > threshold:
            if len(sentence) > 0:
                if actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])
                    
        if len(sentence) > 5:
            sentence = sentence[-5:]
        
        cv2. rectangle(image, (0,0), (640,40), (245,117,16),-1)
        cv2.putText(image, ' '.join(sentence),(3,30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        cv2.imshow('OpenCV Feed',image)
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 33ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 44ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 39ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 43ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 47ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 45ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 57ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 47ms/step
hello
<class 'mediapipe.pyth